# 2 pulses collinear experiment

In [1]:
from __future__ import division, print_function
#from sympy import latex
import sympy as sy
from math import copysign
from operator import add, mul
import numpy as np
from functools import partial

In [2]:
from sig2d.signals import Signal, amplitude, signal, signals_for_order

In [3]:
sy.init_printing()

In [4]:
# utilites and filters
def fundamental_freq(k):
    return sum([copysign(1,i) for i in k]) == 1

#def positive_last(k):
#    return k[-1] > 0

def multifilter(filters, reductop=all):
    def composite(k):
        return reductop([f(k) for f in filters])
    return composite

def accumulator(l, op=add):
    """
    Returns the running accumulation of `l` using operator `op`.
    
    accumulate(l) results in the running sum.
    """
    l = iter(l)
    r = l.next()
    for v in l:
        r = op(r, v)
        yield r
        
def accumulated(l, op=add):
    """Returns an running accumulation of `l` as a list."""
    return list(accumulator(l, op))
        

#def ends_with_k3(k):
#    return abs(k[-1]) == 3

#def single_k3(k):  # emitted in the k3 direction?
#    return sum([abs(i)==3 for i in k]) == 1

# emit in k3 direction
def emit_k3(k):
    # total +1 in k3 direction
    # others cancel out
    return (sum([copysign(1, i) for i in k if abs(i)==3]) == 1) and (sum([copysign(1, i) for i in k if abs(i)!=3]) == 0)

#def rwa(k):  # avoid +k1 +k1 sequences, which yeild |2><0| or try to deexcite the gs
#    # never get more than +1 or -1 total excitation, is that allright? # actually wrong. we can have |2><0| in two steps
#    runsum = accumulated([copysign(1, i) for i in k]) 
#    return all([abs(r) <= 1 for r in runsum])

def contains(s, sig):
    """Signal contains string s"""
    return s in repr(sig)

def contains_filter(s):
    return partial(contains, s)

#pp_2d = multifilter([fundamental_freq, positive_last, single_k3, single_excitation])
filter_collinear = multifilter([fundamental_freq])

## collinear 2 pulses signals, up to 3rd orer

In [5]:
sigs = sum(signals_for_order(i, n_pulses=2, filter_=filter_collinear) for i in [1, 2, 3])
sigs

In [6]:
amps = {i: amplitude(i) for i in [1,2]}
efield = {i+1: v for i, v in enumerate(sy.symbols("E_1:3"))}
detected = sigs + sum(amps[i]*efield[i] for i in sorted(amps.keys()))
detected = sy.expand(sy.conjugate(detected)*detected)
# prepare further neglect
head = detected.func
args = detected.args
# filter signal/signal cross terms
args = filter(contains_filter("E_"), args)
detected = head(*args)
detected

In [7]:
detected_r = detected.args
detected_r = filter(lambda s: not r'conjugate(\chi' in repr(s), detected_r)
detected_r = head(detected_r)

In [8]:
len(detected.args), len(detected_r.args)

## 4x1 phase cycling

In [9]:
phases = [0, sy.pi/2, sy.pi, 3*sy.pi/2]
factors = [1, -1, 1, -1]
sig = sum([f*detected.subs([(amps[1], sy.exp(-sy.I*p)),
                            (amps[2], 1)])
           for f, p in zip(factors, phases)]
    )
sy.re(sig.expand(complex=True))
sig

### with chopping

In [23]:
# prepare chopping:
a = {i+1: v for i, v in enumerate(sy.symbols('a1:3', real=True, nonnegative=True))}
phi = {i+1: v for i, v in enumerate(sy.symbols('\phi_1:3', real=True))}
detected = detected.subs([(amps[i], a[i]*sy.exp(-sy.I*phi[i])) for i in amps.keys()])

In [26]:
phases = [0, sy.pi/2, sy.pi, 3*sy.pi/2]
factors = [1, -1, 1, -1]
sig = sum([f*detected.subs([(phi[1], p)])
           for f, p in zip(factors, phases)]
    )
sy.re(sig.expand(complex=True))
sig

In [39]:
sig.subs([(phi[2], sy.pi/4)]).simplify()

In [41]:
phases = [0, sy.pi/4]
factors = [1, sy.I]
sum([f*sig.subs([(phi[2], p)])
           for f, p in zip(factors, phases)])